In [1]:
# Import pandas for working with tables conveniently
import pandas as pd

### Create and arrange tables for little lemon data base

In [2]:
# Create new table from csv file
main_table = pd.read_csv('LittleLemon_data.csv')

In [3]:
pd.set_option('display.max_columns', None)
main_table.reset_index(drop=True)
main_table.head(3)


,Row Number,Order ID,Order Date,Delivery Date,Customer ID,Customer Name,City,Country,Postal Code,Country Code,Cost,Sales,Quantity,Discount,Delivery Cost,Course Name,Cuisine Name,Starter Name,Desert Name,Drink,Sides
0,1,54-366-6861,6/15/20,3/26/20,72-055-7985,Laney Fadden,Daruoyan,China,993-0031,CN,125.0,187.5,2,20.00,60.51,Greek salad,Greek,Olives,Greek yoghurt,Athens White wine,Tapas
1,2,63-761-3686,8/25/20,7/17/20,65-353-0657,Giacopo Bramich,Ongjin,North Korea,216282,KP,235.0,352.5,1,15.00,96.75,Bean soup,Italian,Flatbread,Ice cream,Corfu Red Wine,Potato salad
2,3,65-351-6434,8/17/21,4/24/20,90-876-6799,Lia Bonar,Quince Mil,Peru,663246,PE,75.0,112.5,3,10.52,36.37,Pizza,Italian,Minestrone,Cheesecake,Italian Coffee,Bruschetta


In [4]:
#Changes the column name Cost to get rid of the leading space that come with the .CSV file
main_table.rename(columns = {' Cost':'Cost'}, inplace = True)
main_table['Cost'].head(3)

0    125.0
1    235.0
2     75.0
Name: Cost, dtype: float64

## Create Orders, Customers, Menus, MenuItems tables from main_table

In [5]:
#Define column names for each table
Customers_columns = ['Customer ID','Customer Name']
Orders_columns = ['Order ID','Customer ID','Cost','Sales','Quantity','Discount','Delivery Cost']
Menus_columns = ['Course Name','Cuisine Name']
MenuItems_columns = ['Course Name','Starter Name','Desert Name','Drink','Sides']


In [6]:
#Populate data into Customers table
Customers = main_table.loc[0:,Customers_columns]
print(len(Customers))

# Sorting by Customer ID
Customers.sort_values('Customer ID', inplace=True)

# Dropping the duplicated Customer ID 
Customers.drop_duplicates(subset='Customer ID',
                     keep='first', inplace=True)

print(len(Customers))
Customers.reset_index(drop=True,inplace=True)
Customers.head(5)

21000
1000


,Customer ID,Customer Name
0,00-090-3491,Bradford Rafe
1,00-352-9063,Sharon Lazare
2,00-381-6823,Eba Harriagn
3,00-435-7006,Silas Asson
4,00-689-5519,Jill O'Sirin


In [7]:
#Populate data into Orders table
#Orders can be duplicated no need to drop
Orders = main_table.loc[0:,Orders_columns]
print(len(Orders))

# Sorting by Order ID
Orders.sort_values('Order ID', inplace=True)

# Dropping the duplicated Order ID 
Orders.drop_duplicates(subset='Order ID',
                     keep='first', inplace=True)

print(len(Orders))
Orders.reset_index(drop=True,inplace=True)
Orders.head(5)

21000
1000


,Order ID,Customer ID,Cost,Sales,Quantity,Discount,Delivery Cost
0,00-008-3271,50-245-9029,168.27,252.405,2,10.00,22.25
1,00-283-3544,07-369-4272,93.13,139.695,2,47.84,45.51
2,00-350-1396,39-459-6791,235.00,352.500,1,10.00,81.13
3,00-451-8857,11-253-6502,210.00,315.000,1,98.01,48.81
4,00-465-7031,84-300-6499,320.00,480.000,3,97.48,95.63


In [8]:
#Populate data into Menus table
Menus = main_table.loc[0:,Menus_columns]
print(len(Menus))

# Sorting by Course name
Menus.sort_values('Course Name', inplace=True)

# Dropping duplicate Course names
Menus.drop_duplicates(subset='Course Name',
                     keep='first', inplace=True)
print(len(Menus))

#Reset index
Menus.reset_index(drop=True,inplace=True)

#Add sourced index to MenuID (no duplicate) and start from 1
Menus['MenuID'] = Menus.index+1

Menus.head()

21000
6


,Course Name,Cuisine Name,MenuID
0,Bean soup,Turkish,1
1,Carbonara,Italian,2
2,Greek salad,Greek,3
3,Kabasa,Italian,4
4,Pizza,Italian,5


In [9]:
#Populate data into MenuItems table
MenuItems = main_table.loc[0:,MenuItems_columns]

#Add sourced index to MenuItemID (no duplicate) and start from 1
MenuItems['MenuItemsID'] = MenuItems.index+1

MenuItems.head(5)

,Course Name,Starter Name,Desert Name,Drink,Sides,MenuItemsID
0,Greek salad,Olives,Greek yoghurt,Athens White wine,Tapas,1
1,Bean soup,Flatbread,Ice cream,Corfu Red Wine,Potato salad,2
2,Pizza,Minestrone,Cheesecake,Italian Coffee,Bruschetta,3
3,Carbonara,Tomato bread,Affogato,Roma Red wine,Focaccia,4
4,Kabasa,Falafel,Turkish yoghurt,Ankara White Wine,Meatballs,5


# Insert data from each table into the table in the database

In [34]:
# Import mysql connector and connect to the data base
import mysql.connector as connector

connection=connector.connect(user="root",password="")

cursor=connection.cursor()
cursor.execute("USE LittleLemonDB;")
cursor.execute("SHOW TABLES;")
results = cursor.fetchall()

print(results)


[('Customers',), ('MenuItems',), ('Menus',), ('Orders',)]


In [11]:
#Insert data into Customers table
for index,column in Customers.iterrows():
    
    insert_query = '''INSERT INTO Customers (CustomerID,CustomerName)
    VALUES
    ("{}","{}");'''.format(column['Customer ID'],column['Customer Name'])
    
    cursor.execute(insert_query)
    connection.commit()


In [12]:
#Insert data into Menus table
for index,column in Menus.iterrows():
    
    insert_query = '''INSERT INTO Menus (CourseName,CuisineName,MenuID)
    VALUES
    ("{}","{}",{});'''.format(column['Course Name'],column['Cuisine Name'],column['MenuID'])
    
    cursor.execute(insert_query)
    connection.commit()

In [13]:
#Insert data into MenuItems table
for index,column in MenuItems.iterrows():
    
    insert_query = '''INSERT INTO MenuItems (CourseName,StarterName,DesertName,Drink,Sides,MenuItemsID)
    VALUES
    ("{}","{}","{}","{}","{}",{});'''.format(column['Course Name'],column['Starter Name'],
                                             column['Desert Name'],column['Drink'],
                                             column['Sides'],column['MenuItemsID'])
    
    cursor.execute(insert_query)
    connection.commit()

In [14]:
#Insert data into Orders table
for index,column in Orders.iterrows():
    
    insert_query = '''INSERT INTO Orders (OrderID,CustomerID,Cost,Sales,Quantity,Discount,DeliveryCost)
    VALUES
    ("{}","{}",{},{},{},{},{});'''.format(column['Order ID'],column['Customer ID'],
                                             column['Cost'],column['Sales'],
                                             column['Quantity'],column['Discount'],
                                             column['Delivery Cost'])
    
    cursor.execute(insert_query)
    connection.commit()

## Check if the data insert to the tables successfully

In [35]:
#Check data in Customers table. Print only first 5 records.
cursor.execute("SELECT * FROM Customers;")
results = cursor.fetchall()
print(cursor.column_names)
for i in range(5):
    print(results[i])
    

('CustomerID', 'CustomerName')
('00-090-3491', 'Bradford Rafe')
('00-352-9063', 'Sharon Lazare')
('00-381-6823', 'Eba Harriagn')
('00-435-7006', 'Silas Asson')
('00-689-5519', "Jill O'Sirin")


In [36]:
#Check data in Menus table. Print only first 5 records.
cursor.execute("SELECT * FROM Menus;")
results = cursor.fetchall()
print(cursor.column_names)
for i in range(5):
    print(results[i])

('CourseName', 'CuisineName', 'MenuID')
('Bean soup', 'Turkish', 1)
('Carbonara', 'Italian', 2)
('Greek salad', 'Greek', 3)
('Kabasa', 'Italian', 4)
('Pizza', 'Italian', 5)


In [37]:
#Check data in MenuItems table. Print only first 5 records.
cursor.execute("SELECT * FROM MenuItems;")
results = cursor.fetchall()
print(cursor.column_names)
for i in range(5):
    print(results[i])

('CourseName', 'StarterName', 'DesertName', 'Drink', 'Sides', 'MenuItemsID')
('Greek salad', 'Olives', 'Greek yoghurt', 'Athens White wine', 'Tapas', 1)
('Bean soup', 'Flatbread', 'Ice cream', 'Corfu Red Wine', 'Potato salad', 2)
('Pizza', 'Minestrone', 'Cheesecake', 'Italian Coffee', 'Bruschetta', 3)
('Carbonara', 'Tomato bread', 'Affogato', 'Roma Red wine', 'Focaccia', 4)
('Kabasa', 'Falafel', 'Turkish yoghurt', 'Ankara White Wine', 'Meatballs', 5)


In [38]:
#Check data in Orders table. Print only first 5 records.
cursor.execute("SELECT * FROM Orders;")
results = cursor.fetchall()
print(cursor.column_names)
for i in range(5):
    print(results[i])

('OrderID', 'CustomerID', 'Cost', 'Sales', 'Quantity', 'Discount', 'DeliveryCost')
('00-008-3271', '50-245-9029', Decimal('168.27'), Decimal('252.41'), 2, Decimal('10.00'), Decimal('22.25'))
('00-283-3544', '07-369-4272', Decimal('93.13'), Decimal('139.70'), 2, Decimal('47.84'), Decimal('45.51'))
('00-350-1396', '39-459-6791', Decimal('235.00'), Decimal('352.50'), 1, Decimal('10.00'), Decimal('81.13'))
('00-451-8857', '11-253-6502', Decimal('210.00'), Decimal('315.00'), 1, Decimal('98.01'), Decimal('48.81'))
('00-465-7031', '84-300-6499', Decimal('320.00'), Decimal('480.00'), 3, Decimal('97.48'), Decimal('95.63'))
